In [11]:
#Get data ready for GLMM for ensemble methods
#copy paste ROC cutoff, logistic regression coefficient from step 4 before running this one

#------------import predictions of GAT and BERT------------------
#These paths are different from it in  
path_result='C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\results_Cmodel\\'
path_data="C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\data\\"
path_code="C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\"

#C:\Backup of covid project\2cls_CScombined_negVSnonneg
import numpy as np 
import urllib.request 
import os 
import csv 
import requests 
import time
import math
import rando
import pandas as pd
from collections import defaultdict
import pickle
import re, string, unicodedata
import nltk
import contractions
import inflect
import heapq
import pickle
import warnings
import random
from sklearn.utils import shuffle

os.chdir(path_data)

types=["comment"]
years=['2019','2020']
schools = ["notredame","uofm","columbia","UCSD","berkeley","Harvard","ucla"]
categ={"notredame":[1,1,1],"uofm":[1,1,0],"columbia":[1,0,1],"dartmouth":[0,1,1],
       "UCSD":[1,0,0],"berkeley":[0,1,0],"Harvard":[0,0,1],"ucla":[0,0,0]}
label_group=[" "]
months=['08','09','10','11']
t_com=['time','id','body','Emotion','emo_pred']
drop_com=["link_id",'Topic',"parent_id","author_fullname","author","gildings","score","subreddit","no_follow","total_awards_received","all_awardings","is_submitter","locked","send_replies","stickied",]
emo=["Very Positive","Positive","Neutral","Negative","Very negative"]

df_all = pd.DataFrame(columns = ['Emotion','predicted_HAN_neg','predicted_HAN_pos','predicted_BERT_neg','predicted_BERT_pos','truth_label','truth_rand','school','year','schoolyear'])
ii=0
for t in types:
    for s in schools:
        for y in years:
            print(ii)
            ii+=1
            f=s+y+'.p'
            os.chdir(path_result)
            predicted=pickle.load(open(f,'rb'))
            from scipy.special import softmax
            m=softmax(predicted,axis=1)
            pred_HAN=m
            
            f="label_emt3_"+s+y+".csv"
            os.chdir(path_data)
            rand_truth1=pd.read_csv(f,skip_blank_lines=True, header=None)#.values
            rand_truth=np.argmax(rand_truth1.values,axis=1)
                
            data = pd.DataFrame(columns = ['Emotion'])
            predicted = pd.DataFrame(columns = ['emo_pred_neg','emo_pred_pos'])
            
            for l in label_group:
                f=s+y+".csv"
                os.chdir(path_data)
                dat=pd.read_csv(f,skip_blank_lines=True)
                data=data.append(dat,ignore_index = True)

                f=s+y+'_score.csv'
                os.chdir(path_data)
                predicted1=pd.read_csv(f,skip_blank_lines=True)
                predicted=predicted.append(predicted1[['emo_pred_neg','emo_pred_pos']],ignore_index = True)
            
            predicted=predicted[['emo_pred_neg','emo_pred_pos']].values
            for i in range(len(predicted)):
                p0=predicted[i,0]
                #predicted[i,0]=p[0]/(p[0]+p[1])
                predicted[i,1]=1-p0
            pred_BERT=predicted

            w=data
            w['predicted_HAN_neg']=pred_HAN[:,0]
            w['predicted_HAN_pos']=pred_HAN[:,1]
            w['predicted_BERT_neg']=pred_BERT[:,0]
            w['predicted_BERT_pos']=pred_BERT[:,1]

            orig_truth=[]
            for e in w['Emotion'].values:
                if e=='Negative' or e=='Very negative':
                    orig_truth.append(0)
                elif e=='Positive' or e=='Very Positive':
                    orig_truth.append(1)
                elif e=='Neutral':
                    orig_truth.append(2)
                else:
                    orig_truth.append(3)
            w['truth_label']=np.array(orig_truth)
            w['truth_rand']=rand_truth
            w['schoolyear']=s+y
            w['school']=s
            w['year']=y
            df_all = df_all.append(w,ignore_index = True)

print(ii)
ii+=1
#append dartmouth to df
s='dartmouth'
f=s+'.p'
os.chdir(path_result)
predicted=pickle.load(open(f,'rb'))
m=softmax(predicted,axis=1)
pred_HAN=m

f=s+'_score.csv'
os.chdir(path_data)
predicted=pd.read_csv(f,skip_blank_lines=True)
predicted=predicted[['emo_pred_neg','emo_pred_pos']].values
for i in range(len(predicted)):
    p0=predicted[i,0]
    #predicted[i,0]=p[0]/(p[0]+p[1])
    predicted[i,1]=1-p0
pred_BERT=predicted

print(1)
f4='dartmouth.csv'
os.chdir(path_data)
data4=pd.read_csv(f4,skip_blank_lines=True)
data4['year']='0'
for d in range(len(data4['id'])):
    data4['year'][d]=str(data4['time'][d])[0:4]
data = data4

print(2)
f="label_emt3_"+s+".csv"
os.chdir(path_data)
rand_truth1=pd.read_csv(f,skip_blank_lines=True, header=None)#.values
rand_truth=np.argmax(rand_truth1.values,axis=1)

print(3)
w=data
w['predicted_HAN_neg']=pred_HAN[:,0]
w['predicted_HAN_pos']=pred_HAN[:,1]
w['predicted_BERT_neg']=pred_BERT[:,0]
w['predicted_BERT_pos']=pred_BERT[:,1]

print(4)
orig_truth=[]
for e in w['Emotion'].values:
    if e=='Negative' or e=='Very negative':
        orig_truth.append(0)
    elif e=='Positive' or e=='Very Positive':
        orig_truth.append(1)
    elif e=='Neutral':
        orig_truth.append(2)
    else:
        orig_truth.append(3)
w['truth_label']=np.array(orig_truth)
w['truth_rand']=rand_truth
w['schoolyear']=s
w['school']=s
df_all = df_all.append(w,ignore_index = True)

#--------------------------------normalize all probability using cut0ff--------------
cutoff_han=0.428274
cutoff_bert=0.692322


df_all['pred_han_neg_norm']=int(0)
df_all['pred_han_pos_norm']=int(0)
df_all['pred_bert_neg_norm']=int(0)
df_all['pred_bert_pos_norm']=int(0)

for d in range(len(df_all['body'])):
    if df_all.iloc[d]['predicted_HAN_pos']>=cutoff_han:
        newpos=0.5*(df_all.iloc[d]['predicted_HAN_pos']-cutoff_han)/(1-cutoff_han)+0.5
    elif df_all.iloc[d]['predicted_HAN_pos']<cutoff_han:
        newpos=0.5*df_all.iloc[d]['predicted_HAN_pos']/cutoff_han
    df_all.iloc[d, df_all.columns.get_loc('pred_han_pos_norm')]=newpos
    df_all.iloc[d, df_all.columns.get_loc('pred_han_neg_norm')]=1-newpos
    
    if df_all.iloc[d]['predicted_BERT_pos']>=cutoff_bert:
        newpos=0.5*(df_all.iloc[d]['predicted_BERT_pos']-cutoff_bert)/(1-cutoff_bert)+0.5
    elif df_all.iloc[d]['predicted_BERT_pos']<cutoff_bert:
        newpos=0.5*df_all.iloc[d]['predicted_BERT_pos']/cutoff_bert
    df_all.iloc[d, df_all.columns.get_loc('pred_bert_pos_norm')]=newpos
    df_all.iloc[d, df_all.columns.get_loc('pred_bert_neg_norm')]=1-newpos
    
    
    if d%5000==0: print(d)
#(df)#.iloc[1])#['predicted_HAN_neg'])
print('df_all finished')

#---------------------------- using logistic regression to get ensemble prediction----
betas=[-2.97772323,2.4231413,3.24225897]

df_all['pred_esm']=0
df_all['pred_HAN']=0
df_all['pred_BERT']=0
schoolyears=["notredame2019","notredame2020","uofm2019","uofm2020","columbia2019","columbia2020","UCSD2019","UCSD2020","berkeley2019","berkeley2020","Harvard2019","Harvard2020","ucla2019","ucla2020","dartmouth"]        
for d in range(len(df_all['schoolyear'])):
    negHAN=df_all['pred_han_neg_norm'].iloc[d]
    posHAN=df_all['pred_han_pos_norm'].iloc[d]
    negBERT=df_all['pred_bert_neg_norm'].iloc[d]
    posBERT=df_all['pred_bert_pos_norm'].iloc[d]
    if posHAN>=0.5:
        #df['pred_HAN'].iloc[d]=0
        df_all.iloc[d, df_all.columns.get_loc('pred_HAN')] =1
    elif posHAN<=0.5:
        #df['pred_HAN'].iloc[d]=1
        df_all.iloc[d, df_all.columns.get_loc('pred_HAN')] =0
    if posBERT>=0.5:
        #df['pred_BERT'].iloc[d]=0
        df_all.iloc[d, df_all.columns.get_loc('pred_BERT')] =1
    elif posBERT<=0.5:
        #df['pred_BERT'].iloc[d]=1
        df_all.iloc[d, df_all.columns.get_loc('pred_BERT')] =0
    log_pos_odds=betas[0]+betas[1]*posHAN+betas[2]*posBERT
    
    df_all.iloc[d, df_all.columns.get_loc('pred_esm')]=int(log_pos_odds>0)



#------------transform data to GLMM 
os.chdir(path_data)
schools = ["notredame","uofm","columbia","dartmouth","UCSD","berkeley","Harvard","ucla"]
categ={"notredame":[1,1,1],"uofm":[1,1,0],"columbia":[1,0,1],"dartmouth":[0,1,1],
       "UCSD":[1,0,0],"berkeley":[0,1,0],"Harvard":[0,0,1],"ucla":[0,0,0]}

#Time period: Aug to Nov 2019 (baseline) and  Aug to Nov. 2020;
#A in person learning 1; others (online, hybrid) 0
#B small city 1, large city 0
#C private 1, public 0
#   A  B  C 
#1  1  1  1  
#2  1  1  0 
#3  1  0  1   
#4  0  1  1
#5  1  0  0   
#6  0  1  0 
#7  0  0  1   
#8  0  0  0  

#1 Notre Dame (in-person learning 1, small city 1, private 1)
#2  Michigan   ( in-person learning  1,  small city 1, public 0)
#3. Columbia   ( in-person  1, large city 0, private 1) 
#4. Dartmouth  (online 0,  small city 1, private 1)
#5  UCSD (in-person learning 1,  large city 0 , public 0)  
#6  xxx   (0 online  small city 1, public 0)  
#7  Harvard    (online 0,  large city 0, private 1)
#8  UCLA (online 0,  large city 0, public 0)

#Positive: 1, negative:0
#2020:1, 2019:0
for d in range(len(df_all["schoolyear"])):
    if (d/1000-int(d/1000))==0:
        print(d)
    y=df_all['year'][d]
    s=df_all['school'][d]
    #sentence=data["body"][d]
    #emb=np.array(use(sentence))[0]
    line=[df_all["pred_esm"][d]]
    line.append(int(y=='2020'))
    if categ[s][0]==1:
        line.append(1)
    elif categ[s][0]==0:
        if y=='2020':
            line.append(0)
        elif y=='2019':
            line.append(1)
    line.append(categ[s][1])
    line.append(categ[s][2])
    line.append(schools.index(s))
    #line=line+emb.tolist()
    name='GLMM_esm.csv'
    os.chdir(path_result)
    e=open(name, 'a',newline='')
    with e:
        writer = csv.writer(e, delimiter=',')
        writer.writerow(line)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1


c:\users\owner\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2
3
4
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
df_all finished
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125

In [12]:
#get GLMM ready for BERT alone and RoBERTa alone
path_result='C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\results_Cmodel\\'
path_data="C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\data\\"
path_code="C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\"

os.chdir(path_data)

types=["comment"]
years=['2019','2020']
#years=['2020']
schools = ["notredame","uofm","columbia","dartmouth","UCSD","berkeley","Harvard","ucla"]
categ={"notredame":[1,1,1],"uofm":[1,1,0],"columbia":[1,0,1],"dartmouth":[0,1,1],
       "UCSD":[1,0,0],"berkeley":[0,1,0],"Harvard":[0,0,1],"ucla":[0,0,0]}

#Time period: Aug to Nov 2019 (baseline) and  Aug to Nov. 2020;
#A in person learning 1; others (online, hybrid) 0
#B small city 1, large city 0
#C private 1, public 0
#   A  B  C 
#1  1  1  1  
#2  1  1  0 
#3  1  0  1   
#4  0  1  1
#5  1  0  0   
#6  0  1  0 
#7  0  0  1   
#8  0  0  0  

#1 Notre Dame (in-person learning 1, small city 1, private 1)
#2  Michigan   ( in-person learning  1,  small city 1, public 0)
#3. Columbia   ( in-person  1, large city 0, private 1) 
#4. Dartmouth  (online 0,  small city 1, private 1)
#5  UCSD (in-person learning 1,  large city 0 , public 0)  
#6  UCB   (0 online  small city 1, public 0)  
#7  Harvard    (online 0,  large city 0, private 1)
#8  UCLA (online 0,  large city 0, public 0)

#Positive: 1, negative:0
#2020:1, 2019:0
att_com=['time','id','body','Emotion','Topic','emo_pred']
emo=["Very Positive","Positive","Neutral","Negative","Very negative"]
top=["Covid","Academics","Sports","Campus/Students Life","Social Media","Religion","Politics","Others"]

#Positive: 1, negative:0
for d in range(len(df_all["schoolyear"])):
    if (d/1000-int(d/1000))==0:
        print(d)
    y=df_all['year'][d]
    s=df_all['school'][d]
    #sentence=data["body"][d]
    #emb=np.array(use(sentence))[0]
    line=[df_all["pred_HAN"][d]]
    line.append(int(y=='2020'))
    if categ[s][0]==1:
        line.append(1)
    elif categ[s][0]==0:
        if y=='2020':
            line.append(0)
        elif y=='2019':
            line.append(1)
    line.append(categ[s][1])
    line.append(categ[s][2])
    line.append(schools.index(s))
    #line=line+emb.tolist()
    name='GLMM_esm_han.csv'
    os.chdir(path_result)
    e=open(name, 'a',newline='')
    with e:
        writer = csv.writer(e, delimiter=',')
        writer.writerow(line)

        
for d in range(len(df_all["schoolyear"])):
    if (d/1000-int(d/1000))==0:
        print(d)
    y=df_all['year'][d]
    s=df_all['school'][d]
    #sentence=data["body"][d]
    #emb=np.array(use(sentence))[0]
    line=[df_all["pred_BERT"][d]]
    line.append(int(y=='2020'))
    if categ[s][0]==1:
        line.append(1)
    elif categ[s][0]==0:
        if y=='2020':
            line.append(0)
        elif y=='2019':
            line.append(1)
    line.append(categ[s][1])
    line.append(categ[s][2])
    line.append(schools.index(s))
    #line=line+emb.tolist()
    name='GLMM_esm_bert.csv'
    os.chdir(path_result)
    e=open(name, 'a',newline='')
    with e:
        writer = csv.writer(e, delimiter=',')
        writer.writerow(line)
#df_all

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [13]:
#format first block for table4 in plosone paper

years=['2019','2020']
methods=['GAT','bert','ens']
schoolyears2019=["ucla2019","UCSD2019","berkeley2019","uofm2019","Harvard2019","columbia2019","dartmouth2019","notredame2019",'overall2019']
schoolyears2020=["ucla2020","UCSD2020","berkeley2020","uofm2020","Harvard2020","columbia2020","dartmouth2020","notredame2020",'overall2020']
schools=["ucla","UCSD","berkeley","uofm","Harvard","columbia","dartmouth","notredame"]
schoolnames=["UCLA","UCSD","UCB","UMich","Harvard","Columbia","Dartmouth","ND"]
neg2019=[]
pos2019=[]
total2019=[]

neg2020=[]
pos2020=[]
total2020=[]
for s in schools:
    for y in years:
        if y=='2019':
            line='\\multirow{2}{5em}{'+schoolnames[schools.index(s)]+'}'+' & 2019 & '
            neg=len(df_all.loc[(df_all.pred_esm==float(0)) & (df_all.school==s) & (df_all.year==(y))]['year'])
            pos=len(df_all.loc[(df_all.pred_esm==float(1)) & (df_all.school==s) & (df_all.year==(y))]['year'])
            total=neg+pos
            neg2019.append(neg)
            pos2019.append(pos)
            total2019.append(total)
        elif y=='2020':
            line=' & 2020 & '
            neg=len(df_all.loc[(df_all.pred_esm==float(0)) & (df_all.school==s) & (df_all.year==(y))]['year'])
            pos=len(df_all.loc[(df_all.pred_esm==float(1)) & (df_all.school==s) & (df_all.year==(y))]['year'])
            total=neg+pos
            neg2020.append(neg)
            pos2020.append(pos)
            total2020.append(total)
        perc='{:.2f}'.format(round(neg/total*100, 2))
        line=line+str(neg)+' & '+str(pos)+' & '+str(total)+' & '+perc+"\\\\"
        print(line)
    print('\\hline')
line='\\multirow{2}{5em}{total}'+' & 2019 & '
neg=sum(neg2019)
pos=sum(pos2019)
total=sum(total2019)
perc='{:.2f}'.format(round(neg/total*100, 2))
line=line+str(neg)+' & '+str(pos)+' & '+str(total)+' & '+perc+"\\\\"
print(line)

line=' & 2020 & '
neg=sum(neg2020)
pos=sum(pos2020)
total=sum(total2020)
perc='{:.2f}'.format(round(neg/total*100, 2))
line=line+str(neg)+' & '+str(pos)+' & '+str(total)+' & '+perc+"\\\\"
print(line)
print('\\hline')

\multirow{2}{5em}{UCLA} & 2019 & 6121 & 10773 & 16894 & 36.23\\
 & 2020 & 7680 & 11034 & 18714 & 41.04\\
\hline
\multirow{2}{5em}{UCSD} & 2019 & 6502 & 11000 & 17502 & 37.15\\
 & 2020 & 6696 & 10650 & 17346 & 38.60\\
\hline
\multirow{2}{5em}{UCB} & 2019 & 6650 & 10023 & 16673 & 39.88\\
 & 2020 & 7699 & 10495 & 18194 & 42.32\\
\hline
\multirow{2}{5em}{UMich} & 2019 & 4126 & 8398 & 12524 & 32.94\\
 & 2020 & 6922 & 7957 & 14879 & 46.52\\
\hline
\multirow{2}{5em}{Harvard} & 2019 & 935 & 1958 & 2893 & 32.32\\
 & 2020 & 1138 & 2416 & 3554 & 32.02\\
\hline
\multirow{2}{5em}{Columbia} & 2019 & 1686 & 3345 & 5031 & 33.51\\
 & 2020 & 3423 & 5914 & 9337 & 36.66\\
\hline
\multirow{2}{5em}{Dartmouth} & 2019 & 308 & 966 & 1274 & 24.18\\
 & 2020 & 284 & 942 & 1226 & 23.16\\
\hline
\multirow{2}{5em}{ND} & 2019 & 395 & 1184 & 1579 & 25.02\\
 & 2020 & 4232 & 3718 & 7950 & 53.23\\
\hline
\multirow{2}{5em}{total} & 2019 & 26723 & 47647 & 74370 & 35.93\\
 & 2020 & 38074 & 53126 & 91200 & 41.75\\
\hline
